# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным и сравнить его эффективность с ансамблями из самых популярных библиотек.

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.utils import resample
import matplotlib
import copy
import numpy as np
import pandas as pd
from collections import Counter
from tqdm import tqdm
from sklearn.metrics import accuracy_score, recall_score, precision_score
from statistics import mode
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from itertools import product
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier

Реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание. Используйте реализацию дерева из HW3.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

Возьмем дерево из sklearn для ускорения работы случайного леса.

In [ ]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=3, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.n_estimators = n_estimators
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        
        # для подсчета out-of-bag
        self.err_oob = []
        self.err_oob_j = {}
        
        # будем хранить построенные деревья в списке
        self.forest = []
    
    def fit(self, X, y):
        for n in range(self.n_estimators):
            tree = DecisionTreeClassifier(max_depth=self.max_depth, max_features="auto",
                                          min_samples_leaf=self.min_samples_leaf, criterion=self.criterion)
            X_fit, y_fit = resample(X, y)
            tree.fit(X_fit, y_fit)
            self.forest.append(tree)
        
            err_oob_idx = set(X.index) - set(X_fit.index)
            self.err_oob.append(accuracy_score(y.loc[err_oob_idx], tree.predict(X.loc[err_oob_idx])))
            
            # Перемешать значения признака j у объектов выборки
            for col in X.columns:
                X_err = X.loc[err_oob_idx].copy()
                X_err[col] = np.array(resample(X_err[col]))
                
                if col not in self.err_oob_j:
                    self.err_oob_j[col] = [accuracy_score(y.loc[err_oob_idx], tree.predict(X_err))]
                    
                else:
                    self.err_oob_j[col].append(self.err_oob[-1] - accuracy_score(y.loc[err_oob_idx], tree.predict(X_err)))
    
    def predict(self, X_test):
        forest_predicts = np.array([tree.predict(X_test) for tree in self.forest]).T
        forest_proba =  np.array([tree.predict_proba(X_test) for tree in self.forest]).T
        mode_choise = []
        
        for tree in forest_predicts:
            choise = []
            tr = Counter(tree)
            choise.append(max(tr, key=tr.get))
            mode_choise.append(max(choise))
        return mode_choise
    
    def importance(self, X):
        temp = {}
        
        for key in self.err_oob_j:
            temp[key] = sum(self.err_oob_j[key]) / len(self.err_oob_j[key])
            
        return sorted(list(temp.items()), key=lambda x: x[1], reverse=True)

### 
Оптимизируйте по `AUC` на кроссвалидации (размер валидационной выборки - 20%) параметры своей реализации `Random Forest`: 

максимальную глубину деревьев из [2, 3, 5, 7, 10], количество деревьев из [5, 10, 20, 30, 50, 100]. 

Постройте `ROC` кривую (и выведите `AUC` и `accuracy`) для лучшего варианта.

Подсказка: можно построить сразу 100 деревьев глубины 10, а потом убирать деревья и
глубину.

In [ ]:
df = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/x_spam_train.csv')
X = df.drop(columns='Id')
y = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/y_spam_train.csv')
y = y['Expected']
p = {'max_depth' : [2, 3, 5, 7, 10], 'n_estimators' : [5, 10, 20, 30, 50, 100]}
forest = RandomForestClassifier()
keys = p.keys()
iters = product(*p.values())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
for param in iters:
    forest.__init__(**{key : value for key, value in zip(keys, param)})
    forest.fit(X_train, y_train)
    print(param, roc_auc_score(y_test, forest.predict(X_test)))

В качестве параметров возьмем глубину 3 и количество деревьев 40.

In [ ]:
df = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/x_spam_train.csv')
X = df.drop(columns='Id')
y = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/y_spam_train.csv')
y = y['Expected']
forest = RandomForestClassifier(max_depth=3, n_estimators=40)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
forest.fit(X_test, y_test)

In [ ]:
def plot_roc_curve(y_test, p_pred):
    fpr, tpr, thresholds = roc_curve(y_test, p_pred)
    plt.figure(figsize = (7, 7))
    plt.plot(fpr, tpr)
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.xlim(-0.01, 1.01)
    plt.ylim(-0.01, 1.01)
    plt.tight_layout()
    plt.show()
    
y_pred =  forest.predict(X_test)

In [ ]:
plot_roc_curve(y_test, y_pred)
print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest выглядит так:
1. Посчитать out-of-bag ошибку предсказания `err_oob` (https://en.wikipedia.org/wiki/Out-of-bag_error)
2. Перемешать значения признака `j` у объектов выборки (у каждого из объектов изменится значение признака `j` на какой-то другой)
3. Посчитать out-of-bag ошибку (`err_oob_j`) еще раз.
4. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [ ]:
def feature_importance(rfc):
    return rfc.importance(X)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Протестируйте решение на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [ ]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
y = pd.DataFrame(y, columns = ['Expected'])
X = pd.DataFrame(X)

rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y['Expected']))
print("Importance:", feature_importance(rfc))

Проверьте, какие признаки важны для датасета spam? (Используйте файлы x_spam_train и y_spam_train)

_Ваш ответ_

In [ ]:
df = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/x_spam_train.csv')
X = df.drop(columns='Id')
y = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/y_spam_train.csv')
y = y['Expected']
rfc = RandomForestClassifier(n_estimators=3)
rfc.fit(X, y)
print("Importance:", feature_importance(rfc))

1. Обучите модель на всех данных из x_spam_train и y_spam_train.
2. Сделайте submit своего решения и получите значение f1_score не менее 0.6

In [ ]:
# кросс-валидация

df = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/x_spam_train.csv')
X = df.drop(columns='Id')
y = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/y_spam_train.csv')
y = y['Expected']

n = len(y)
n1 = int(np.ceil(n / 3))
n2 = int(np.ceil( 2 * n / 3))
n3 = n

forest = RandomForestClassifier(max_depth=3, n_estimators=40)

forest.fit(X.loc[range(0, n2)], y.loc[range(0, n2)])
y1 = forest.predict(X.loc[range(n2, n3)])
print(accuracy_score(y.loc[range(n2, n3)], y1))

x2 = X.loc[range(n1, n2)]

index = list(set(X.index) - set(x2.index))

forest.fit(X.loc[index], y.loc[index])
y2 = forest.predict(X.loc[range(n1, n2)])
print(accuracy_score(y.loc[range(n1, n2)], y2))
      
forest.fit(X.loc[range(n2, n3)], y.loc[range(n2, n3)])
y3 = forest.predict(X.loc[range(0, n1)])
print(accuracy_score(y.loc[range(0, n1)], y3))

# обучение на всех данных

forest.fit(X, y)

In [ ]:
df = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/x_spam_train.csv')
X = df.drop(columns='Id')
y = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/y_spam_train.csv')
y = y['Expected']
X_spam_test = pd.read_csv('/kaggle/input/homework-ensembles-ib-22/x_spam_test.csv')
submission = pd.DataFrame(columns = ["Id", "Expected"])
submission["Id"] = X_spam_test['Id']
X_spam_test = X_spam_test.drop(columns='Id')
test = forest.predict(X_spam_test)
submission["Expected"] = test
submission.to_csv('submission.csv', index=False)

В качестве альтернативы попробуем библиотечные реализации ансамблей моделей. 

1. [CatBoost](https://catboost.ai/docs/)
2. [XGBoost](https://xgboost.readthedocs.io/en/latest/)
3. [LightGBM](https://lightgbm.readthedocs.io/en/latest/)


Установите необходимые библиотеки. 
Возможно, потребуется установка дополнительных пакетов.

1. Примените модели для нашего датасета.

2. Для стандартного набора параметров у каждой модели нарисуйте `ROC` кривую и выведите `AUC` и `accuracy`.

3. Посчитайте время обучения каждой модели (можно использовать [timeit magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)).

4. Сравните метрики качества и скорость обучения моделей. Какие выводы можно сделать?

_Ваш ответ_

In [ ]:
%%time
from catboost import CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95)
clf = CatBoostClassifier(silent=True)
clf.fit(X_train, y_train)
y_pred =  clf.predict(X_test)
plot_roc_curve(y_test, y_pred)
print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

In [ ]:
%%time
from xgboost import XGBClassifier
clf = XGBClassifier()
clf.fit(X_train, y_train)
y_pred =  clf.predict(X_test)
plot_roc_curve(y_test, y_pred)
print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

In [ ]:
%%time
from lightgbm import LGBMClassifier
clf = LGBMClassifier()
clf.fit(X_train, y_train)
y_pred =  clf.predict(X_test)
plot_roc_curve(y_test, y_pred)
print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))
print('F1:', f1_score(y_test, y_pred))

CatBoostClassifier дольше всех обучается, но имеет самую высокую метрику F1. В целом все три модели достаточно похожи